#### 랜덤 포레스트

In [1]:
# dt의 앙상블 -> 랜덤 포레스트
# 각 dt의 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from sklearn.model_selection import train_test_split

In [13]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [15]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [17]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [39]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [41]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [43]:
rf = RandomForestClassifier(n_jobs = -1, random_state = 42) # rf는 기본적으로 100갸의 결정 트리 사용 -> n_jobs = -1을 서ㄹ정하여 모든 CPU 코어 사용

In [45]:
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1) 
# return_train_score = True -> 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환 (기본값은 False)

In [53]:
np.mean(scores['train_score']) # 훈련 점수

0.9973541965122431

In [55]:
np.mean(scores['test_score']) # 검증 점수

0.8905151032797809

In [57]:
rf.fit(train_input, train_target)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [61]:
rf.feature_importances_ # 변수 중요도

array([0.23167441, 0.50039841, 0.26792718])

In [63]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
# oob_score = 부트스트렙에 포함되지 않는 데이터들을 활용해 모델 평가

In [65]:
rf.fit(train_input, train_target)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [67]:
rf.oob_score_

0.8934000384837406

#### 엑스트라 트리

In [91]:
# 엑스트라 트리는 랜덤 포레스트와 달리 부트스트랩 샘플을 사용하지 않는다 -> 훈련할 때 전체 훈련 세트를 사용
# 노드를 분할할 때 무작위로 분할 splitter = 'random' -> 과적합 방지, 빠른 계산 속도

In [75]:
from sklearn.ensemble import ExtraTreesClassifier

In [77]:
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)

In [81]:
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

In [87]:
np.mean(scores['train_score'])

0.9974503966084433

In [89]:
np.mean(scores['test_score'])

0.8887848893166506

In [93]:
et.fit(train_input, train_target)

ExtraTreesClassifier(n_jobs=-1, random_state=42)

In [95]:
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

#### 그레이디언트 부스팅

In [98]:
# 깊이가 얕은 결정 트리 사용 -> 기본적으로 깊이가 3인 100개의 결정 트리 사용 -> 과적합 방지
# 경사하강법을 사용, 분류에서는 로지스틱 손실 함수, 회귀에서는 평균 제곱 오차 함수 사용

In [100]:
from sklearn.ensemble import GradientBoostingClassifier

In [102]:
gb = GradientBoostingClassifier(random_state = 42)

In [106]:
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

In [110]:
np.mean(scores['train_score'])

0.8881086892152563

In [112]:
np.mean(scores['test_score'])

0.8720430147331015

In [114]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
# n_setimators = 결정 트리 개수
# learning_rate = 학습율

In [116]:
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

In [118]:
np.mean(scores['train_score'])

0.9464595437171814

In [120]:
np.mean(scores['test_score'])

0.8780082549788999

In [122]:
gb.fit(train_input, train_target)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=500, random_state=42)

In [124]:
gb.feature_importances_

array([0.15882696, 0.6799705 , 0.16120254])

#### 히스토그램 기반 그레디언트 부스팅

In [127]:
# 그레디언트 부스틍인 순서대로 트리를 추가해 훈련 속도가 느림 (n_jobs 매개 변수가 없음) -> 속도를 개선한 히스토그램 기반 부스팅

In [129]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [131]:
hgb = HistGradientBoostingClassifier(random_state = 42)

In [137]:
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)

In [139]:
np.mean(scores['train_score'])

0.9321723946453317

In [141]:
np.mean(scores['test_score'])

0.8801241948619236

In [145]:
from sklearn.inspection import permutation_importance # 특성 중요도 계산

In [147]:
hgb.fit(train_input, train_target)

HistGradientBoostingClassifier(random_state=42)

In [151]:
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, # 랜덤하게 섞을 횟수
                                random_state = 42, n_jobs = -1)

In [153]:
result.importances_mean

array([0.08876275, 0.23438522, 0.08027708])

In [155]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, # 랜덤하게 섞을 횟수
                                random_state = 42, n_jobs = -1)

In [157]:
result.importances_mean

array([0.05969231, 0.20238462, 0.049     ])

In [159]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost, LightGBM

In [162]:
from xgboost import XGBClassifier

In [164]:
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)

In [166]:
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)

In [170]:
np.mean(scores['train_score'])

0.9558403027491312

In [172]:
np.mean(scores['test_score'])

0.8782000074035686

In [174]:
from lightgbm import LGBMClassifier

In [176]:
lgb = LGBMClassifier(random_satae = 42)

In [178]:
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)

In [180]:
np.mean(scores['train_score'])

0.935828414851749

In [182]:
np.mean(scores['test_score'])

0.8801251203079884